In [ ]:
# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
# Pandas
import pandas as pd
# RegEx / Regular Expressions
import re

In [213]:
testes = ["DEDO CARACOL 1123.464.100.00-VV-0653-01", "DEDO CARACOL 132.0699C1 440.430A1VV-3379", "DEDO CARACOL 3.099.689 VV-0444-00", "DEDO CARACOL 334.972 - VV-0183-01", "DEDO CARACOL 849.80892 - VV-0252-01", "DEDO CARACOL 849.96278 - VV-0278-01", "DEDO CARACOL 849.96279 - VV-0279-01", "DEDO CARACOL H-102.724-VV-0644-01", "DEDO CARACOL H-169.914 - VV-0214-01", "DEDO DUPLO 065.327-VV-0016-02", "DEDO DUPLO 1.307.299.C2 - VV-0060-00", "DEDO DUPLO 3.318.953/379.720 VV-0003-01", "DEDO DUPLO 466.789-VV-0661", "DEDO DUPLO 466.845-VV-0662", "DEDO DUPLO 475.06883 / VV-4057-00", "ANEL SEGURANCA 331.160 ORIG" ]

vvProdutos = 'https://www.vv.com.br/pt/produto/detalhe/'
codigosVV= []

In [101]:
query = pd.read_excel('vv.xlsx')
display(query)

,ID,Produto,Cod. EAN,Cod. Cotripal
0,1,ACOPLAMENTO 445.956/840.77841 VV-3036-01,7.895824e+12,101290447
1,2,ARRASTADOR CQ-131.05 VV-3195-00,7.895823e+12,101281803
2,3,BARRA CORTE 17 PES 849.92283-VV-0317-01,7.895985e+12,100207194
3,4,BARRA CORTE 30 PES ESQ VV-3183 / VV-6430,7.895985e+12,102566558
4,5,BARRA FIXA 849.92436 VV-0344-01,7.895985e+12,100207200
5,6,BUCHA 58.3380-VV-0372-00,1.824850e+05,100027266
6,7,CATRACA 280.058 - VV-0382-00,1.495490e+05,100021257
7,8,CUBO 320.053 - 373.929 - VV-0393-00,7.895823e+12,101281421
8,9,DEDO DUPLO 466.789-VV-0661,5.732760e+05,100040524
9,10,DEDO TRIPLO 475.06882 / VV-4056-00,NaN,105254575


In [207]:
def buscaInfo(nav, codigo, index):
    if '-' not in codigo or '/' in codigo:
        return
    keys = []
    values = []
    nav.get(str(vvProdutos + codigo))
    nav.implicitly_wait(0.8)
    
    element = nav.find_elements(By.CLASS_NAME, 'itensEquipamentos')
    
    if not element:
        return
    
    for elemento in element:
        ps = elemento.find_elements(By.TAG_NAME, 'p')
        p = []
        for elemento in ps:
            p.append(elemento.text)
        equips = { index : { p[0] : { 'Código OEM' : p[1], 'Equipamentos': p[2:] } } }
    
    keys.append('ID:')
    values.append(index)
    
    element = nav.find_element(By.CLASS_NAME, 'subtitulo')
    nome = element.find_element(By.TAG_NAME, 'h2')
    
    keys.append('Nome:')
    values.append(nome.text)
    
    element = nav.find_element(By.CLASS_NAME, 'descricao')
    li = element.find_elements(By.TAG_NAME, 'li')    
    
    for elemento in li:
        for k in elemento.find_elements(By.TAG_NAME, 'h3'):
            keys.append(k.text)
        for v in elemento.find_elements(By.TAG_NAME, 'p'):
            values.append(v.text)
    dict_infos = dict(zip(keys, values))
    print(dict_infos)
    print(' ')
    
    element = nav.find_element(By.CLASS_NAME, 'listaGaleria')
    imgs = element.find_elements(By.TAG_NAME, 'img')
    src = []
    for img in imgs:
        src.append(img.get_attribute('src'))
    images = {index : src}
    
    print(images)
    
    print(100*'*')
    print(equips)
        

In [208]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('headless')
nav = webdriver.Chrome(options=chrome_options)
produtos = dict(zip(query['ID'], query['Produto']))
for chav, cod in produtos.items():
    i = cod.find('VV')
    produto = cod[i:]
    produto = produto.replace(" ", "")
    produto = produto.replace("VV-", "")
    produtos.update({chav:produto})
for index, codigo in produtos.items():
    try:
        buscaInfo(nav, codigo, index)
    except:
        pass
    print(125*'#')
    
    
    
    
# dict_test = dict(zip(keys, values))

{'ID:': 1, 'Nome:': 'Luva estriada – 3036-01', 'Descrição': 'Luva estriada', 'Linha': 'Colheitadeira', 'Família': 'Bucha', 'Aplicação': 'Tração', 'Material': 'Aço ABNT 4140', 'Tratamento': 'Indução', 'Dureza': '51 a 57 HRc, na região estriada, profundidade 0,8 a 1,6 mm', 'Acabamento': 'Pintura amarela', 'Peso líquido (kg)': '1,3790', 'Peso bruto (kg)': '1,3840'}
 
{1: ['https://www.vv.com.br/admin/adm_autoThumb.php?w=273&h=273&src=media/uploads/produtos/3036-01-1.png', 'https://www.vv.com.br/admin/adm_autoThumb.php?w=273&h=273&src=media/uploads/produtos/3036-01.png']}
****************************************************************************************************
{1: {'New Holland': {'Código OEM': '445956, 84077841', 'Equipamentos': ['CR9080', 'TC5070', 'CS660', 'CR980', 'CR9060', 'TC5040', 'TC5090', 'TC59']}}}
#############################################################################################################################
{'ID:': 2, 'Nome:': 'Arrastador – 3195-00', 'De

{'ID:': 15, 'Nome:': 'Engrenagem – 3459-00', 'Descrição': 'Engrenagem', 'Atributos': 'Z8 P 41,4 mm', 'Linha': 'Colheitadeira', 'Família': 'Engrenagem', 'Aplicação': 'Elevador de grãos', 'Material': 'Ferro fundido cinzento GG25', 'Tratamento': 'Indução', 'Dureza': '40 a 50 HRc', 'Acabamento': 'Pintura verde', 'Peso líquido (kg)': '1,4480', 'Peso bruto (kg)': '1,4480'}
 
{15: ['https://www.vv.com.br/admin/adm_autoThumb.php?w=273&h=273&src=media/uploads/produtos/3459-00.png']}
****************************************************************************************************
{15: {'John Deere': {'Código OEM': 'H164796', 'Equipamentos': ['S690', 'S680', 'S670', 'S660', 'S550', 'S540', '9770STS', '9760STS', '9750STS', '9670STS', '9660STS', '9650STS', '9570STS', '9560STS', '9470STS', '9650', '9550', '9450']}}}
#############################################################################################################################
{'ID:': 16, 'Nome:': 'Conjunto facas – 0352-00', 'Descriç

In [138]:
print(len(src))

32


In [186]:
for teste in testes:
    print(teste)
    indice = teste.find('VV')
    remover = teste[indice:]
    teste = teste.replace(remover, "")
    num = re.search(r'\d+', teste)
    indice = teste.find(num.group())
    remover = teste[:indice]
    teste = teste.replace(remover, "")
    teste = teste.replace('.', "")
    teste = teste.replace('-', "")
    teste = teste.replace(' ', "/")
    teste = teste.split('/')    
#     teste = re.sub('[^0-9]', '', teste)
    print(teste)
    for t in teste:
        if t != '':
            print(t)

DEDO CARACOL 1123.464.100.00-VV-0653-01
['112346410000']
112346410000
DEDO CARACOL 132.0699C1 440.430A1VV-3379
['1320699C1', '440430A1']
1320699C1
440430A1
DEDO CARACOL 3.099.689 VV-0444-00
['3099689', '']
3099689
DEDO CARACOL 334.972 - VV-0183-01
['334972', '', '']
334972
DEDO CARACOL 849.80892 - VV-0252-01
['84980892', '', '']
84980892
DEDO CARACOL 849.96278 - VV-0278-01
['84996278', '', '']
84996278
DEDO CARACOL 849.96279 - VV-0279-01
['84996279', '', '']
84996279
DEDO CARACOL H-102.724-VV-0644-01
['102724']
102724
DEDO CARACOL H-169.914 - VV-0214-01
['169914', '', '']
169914
DEDO DUPLO 065.327-VV-0016-02
['065327']
065327
DEDO DUPLO 1.307.299.C2 - VV-0060-00
['1307299C2', '', '']
1307299C2
DEDO DUPLO 3.318.953/379.720 VV-0003-01
['3318953', '379720', '']
3318953
379720
DEDO DUPLO 466.789-VV-0661
['466789']
466789
DEDO DUPLO 466.845-VV-0662
['466845']
466845
DEDO DUPLO 475.06883 / VV-4057-00
['47506883', '', '', '']
47506883


In [217]:
def contains_number(string):
    return any(char.isdigit() for char in string)

In [222]:
def findOEM(text):
    if 'VV' in text:
        indice = text.find('VV')
        remover = text[indice:]
        text = text.replace(remover, "")
    num = re.search(r'\d+', text)
    indice = text.find(num.group())
    remover = [text[:indice], '.', '-']
    for rm in remover:
        text = text.replace(rm, "")
    text = text.replace(' ', "/")
    text = text.split('/')
    lista = []
    for txt in text:
        if txt:
            lista.append(txt)
    for item in lista:
        if not contains_number(item):
            lista.remove(item)
    print(lista)

In [225]:
settest = 'MOLA 3.099.370 VV-0947'
for teste in testes:
    findOEM(teste)

['112346410000']
['1320699C1', '440430A1']
['3099689']
['334972']
['84980892']
['84996278']
['84996279']
['102724']
['169914']
['065327']
['1307299C2']
['3318953', '379720']
['466789']
['466845']
['47506883']
['331160']


In [224]:
findOEM('APOIO ORIGINAL 821.168 - 370.950')

['821168', '370950']


In [223]:
findOEM('ANEL VEDANTE 474.65268 ORIG')

['47465268']
